# Differential Privacy and GANs in Practice

## Privacy

### Privacy through Aggregation

In this section, we will use a database that has been set up to mandate aggregation. We will see that this is not sufficient to protect individual privacy.

The database has one table called `people` which has columns `age`, `income`, and `zip`.

In [ ]:
from private_db import query_aggregate, query_restricted

`query_aggregate` requires that the query use one of several aggregation functions in SQL, such as average, count, min, max, or sum.

If you haven't used SQL before, don't worry. We won't be using anything too fancy, and the queries will read much like English. You should be able to modify the example queries we give.

Let's get the average income of people who are 50 years old:

In [ ]:
query_aggregate('SELECT AVG(income) FROM people WHERE age=50')

Now let's get the minimum income of people over the age of 50 who live in the `60609` zip code:

In [ ]:
query_aggregate('SELECT MIN(income) FROM people WHERE age>50 AND zip=60609')

### Attack aggregation

Can you see a potential issue with this system?
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

Suppose you know that there is only one 99-year-old living in the 60637 zip code. You can recover their income:

In [ ]:
query_aggregate('SELECT AVG(income) FROM people WHERE age=99 AND zip=60637')

Now consider `query_restricted`. This requires that we aggregate over at least 50 people. Otherwise, it will refuse to return an answer to our query:

In [ ]:
query_restricted('SELECT AVG(income) FROM people WHERE age=99 AND zip=60637')

It turns out in this case we can still recover an individual's income as in the previous case. You know there is only one 98-year-old living in the 60616 zip code. Find their income only using `query_restricted`.

In [ ]:
# your code here

## Gaussian mechanism

Define a function `query_private` which uses the Gaussian mechanism to enforce privacy. Play around with different values of `sigma` ($\sigma$) and see how the attack from above holds up against them. Assume attackers have access to exact population counts for each zip code. How much noise do you need for `SUM` queries? What about `AVG` queries? You can use `np.random.normal` to generate Gaussian noise.

In [ ]:
import numpy as np

# your code here

## Generative adversarial networks

In this section we will show how to code a classic GAN in PyTorch and use it to generate categorical data. You will then have an opportunity to modify the classic GAN into a Wasserstein GAN, and then add modifications so that the GAN satisfies differential privacy. First we must import some packages to implement our GAN and work with data.

In [ ]:
import matplotlib.pyplot as plt # plotting to see how our synthetic data compares to real data
import pandas as pd # data frames for easier manipulation of data
import seaborn as sns # create heatmaps to visualize data
import torch # PyTorch
import torch.nn.functional as F # useful functions in PyTorch

from layers import MultiCategoryGumbelSoftmax # internal class for multicategory outputs
from sklearn.preprocessing import OneHotEncoder # convert our data into a format PyTorch can understand

Here is the simple data frame of artificial flight data we will be trying to reproduce:

In [ ]:
df = pd.DataFrame(
    {'weather': ['sunny']*10000+['cloudy']*10000+['rainy']*10000,
     'status': ['on time']*8000+['delayed']*2000
     + ['on time']*3000+['delayed']*5000+['canceled']*2000
     + ['on time']*1000+['delayed']*3000+['canceled']*6000}
)

In [ ]:
def show_heatmap(df):
    crosstab = pd.crosstab(df.weather, df.status)
    releveled = crosstab.loc[['sunny', 'cloudy', 'rainy'], ['on time', 'delayed', 'canceled']]
    sns.heatmap(releveled, cmap="YlGnBu")
    plt.show()

show_heatmap(df)

You can see that on sunny days, flights are almost always on time, occassionally delayed, and never canceled. On cloudy days flights are more likely to be delayed, and occasionally canceled. On rainy days flights are sometimes delayed and frequently cancelled. Our goal will be to produce a network that can generate data showing these same correlations.

### GAN architecture refresher

Remember that a GAN requires two networks: a generator, which will take random noise as input to produce data that looks like real data, and a discriminator, which will take data and assign a probability that it is real versus fake.

### Generating categorical data

Our example data set uses categorical data with multiple categories. You may have seen `softmax` used before in multiclass classification problems. A classical softmax returns continuous values, which we can interpret as probabilities over the output categories. However, we want to generate data that looks like our real data, which is categorical. We need to get categorical outputs from a softmax, and we need this mechanism to still be differentiable so we can train our network. The Gumbel softmax will allow us to do this, and fortunately it's built into `PyTorch`.

In [ ]:
values = torch.tensor([-1., 0., 1.])

In [ ]:
F.softmax(values, dim=0)

Classical softmax returns continuous values that sum to 1.

In [ ]:
F.gumbel_softmax(values, hard=True)

Gumbel softmax returns a categorical value (when `hard` is set to `True`).

We also must convert our categorical data into a numerical encoding that `PyTorch` can work with. We will use a one-hot encoding. If you haven't seen one-hot encodings before, we have a column for each category level of each variable, and for each observation, we put a 1 in the column for its corresponding category level. `PyTorch` doesn't have ideal tools for this, but we can use `scikit-learn` instead. The example data frame below shows a one-hot encoding for four observations, a sunny on-time flight, a cloudy on-time flight, a rainy canceled flight, and a cloudy delayed flight.

In [ ]:
# example one-hot data frame
pd.DataFrame({'sunny': [1, 0, 0, 0], 'cloudy': [0, 1, 0, 1], 'rainy': [0, 0, 1, 0], 'on time': [1, 1, 0, 0], 'delayed': [0, 0, 0, 1], 'canceled': [0, 0, 1, 0]})[['sunny', 'cloudy', 'rainy', 'on time', 'delayed', 'canceled']]

In [ ]:
# convert data frame to one-hot encoding using sklearn (this will also let us easily convert back to our categories)
enc = OneHotEncoder()
enc.fit(df)
data = torch.tensor(enc.transform(df).toarray()).float()

# the multi category Gumbel softmax needs to know the dimensions of each output variable
output_dims = [len(cat) for cat in enc.categories_]

### Classic GAN

First, let's define some parameters that we will use for our architecture:

In [ ]:
noise_dim = 8 # number of dimensions for noise input to generator
data_dim = data.shape[1] # number of dimensions of the data inputs
hidden_dim = 16 # number of dimensions for the hidden layers

Let's define the architecture of our generator. `torch.nn.Sequential` lets us combine the layers of our network. The first layer will be `torch.nn.Linear`, which is a fully connected layer. The first argument is the input dimension, and the second argument is the output dimension. Since we will pass the noise vector as the input to the generator, the input dimension is `noise_dim`. The output will be `hidden_dim`, for our hidden layer. We then add a non-linearity, in this case a rectified linear unit. Finally, we pass this output through our `MultiCategoryGumbelSoftmax`, which will output categorical data in the shape of our real data.

In [ ]:
generator = torch.nn.Sequential(
    torch.nn.Linear(noise_dim, hidden_dim),
    torch.nn.ReLU(),
    MultiCategoryGumbelSoftmax(hidden_dim, output_dims)
)

Now let's define the architecture of our discriminator. The discriminator takes data as input (real or fake), so the input will have `data_dim`. As with the generator, we use a fully connected layer, followed by a leaky ReLU (don't worry too much about this, people have found this works slightly better than a classical ReLU for the discriminator but not for the generator, but it doesn't make a big difference). Because the discriminator will return a probability (representing the probability it thinks the data is real), we map the output of the ReLU down to a single dimension, and then pass that through a sigmoid to map the value onto the range [0, 1].

In [ ]:
discriminator = torch.nn.Sequential(
    torch.nn.Linear(data_dim, hidden_dim),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(hidden_dim, 1),
    torch.nn.Sigmoid()
)

Here we reach the key part of the GAN, the training loop. There is a lot going on here, especially if you haven't worked with PyTorch before. Everything is commented, and we'll also go over the key parts that make this a GAN specifically.

In [ ]:
def train(data, generator, discriminator,
          epochs=20, n_discriminator=5, batch_size=128,
          learning_rate=1e-3):
    """Train the GAN
    
    Parameters
    ----------
    data : torch.Tensor
        Data for training
    generator : torch.nn.Sequential
        Generator network
    discriminator : torch.nn.Sequential
        Discriminator network
    epochs : int
        Number of iterations over the full data set for training
    n_discriminator : int
        Number of discriminator training iterations
    batch_size : int
        Number of training examples per inner iteration
    learning_rate : float
        Learning rate for training
    """
    # these solvers are optimizers for learning the parameters for our networks
    # we have one for each network (the generator and the discriminator)
    # RMSprop is one of many optimizers, and the choice is not terribly important here
    # The learning rate influences how large of a step the optimizer takes when it updates the parameters
    generator_solver = torch.optim.RMSprop(
        generator.parameters(), lr=learning_rate
    )
    
    discriminator_solver = torch.optim.RMSprop(
        discriminator.parameters(), lr=learning_rate
    )

    # There is a batch for each discriminator training iteration,
    # so each epoch is epoch_length iterations, and the total number of
    # iterations is the number of epochs times the length of each epoch.
    epoch_length = len(data) / (n_discriminator * batch_size)
    n_iters = int(epochs * epoch_length)
    
    # ones and zeros for calculating cross-entropy
    ones_label = torch.ones(batch_size, 1) # the real data should all be labeled 1
    zeros_label = torch.zeros(batch_size, 1) # the fake data should all be labeled 0
    
    # training loop
    for iteration in range(n_iters):
        for _ in range(n_discriminator):
            # Sample real data
            rand_perm = torch.randperm(data.size(0))
            real_sample = data[rand_perm[:batch_size]]

            # Sample fake data
            noise = torch.randn(batch_size, noise_dim)
            fake_sample = generator(noise)

            # Have the discriminator score the data
            discriminator_real = discriminator(real_sample)
            discriminator_fake = discriminator(fake_sample)

            # Calculate discriminator loss
            discriminator_loss = (
                # cross-entropy for real data (all should be 1 for ideal discriminator)
                F.binary_cross_entropy(discriminator_real, ones_label) +
                # cross-entropy for fake data (all should be 0 for ideal discriminator)
                F.binary_cross_entropy(discriminator_fake, zeros_label)
            )

            discriminator_loss.backward() # backpropagate the loss through the discriminator network
            discriminator_solver.step() # update the discriminator network parameters using the optimizer

            # Reset the gradients
            generator.zero_grad()
            discriminator.zero_grad()

        # Sample and score fake data
        noise = torch.randn(batch_size, noise_dim)
        fake_sample = generator(noise)
        discriminator_fake = discriminator(fake_sample)

        # Calculate generator loss
        # generator wants to trick the discriminator into thinking all of the fake data is real (labeled 1)
        generator_loss = F.binary_cross_entropy(discriminator_fake, ones_label)

        generator_loss.backward() # backpropagate the loss through the generator network
        generator_solver.step() # update the generator network parameters using the optimizer

        # Reset the gradients
        generator.zero_grad()
        discriminator.zero_grad()

        # Show training losses and sample crosstabs after each epoch
        if int(iteration % epoch_length) == 0:
            epoch = int(iteration / epoch_length)
            print('Epoch {}\n'
                  'Discriminator loss: {}; '
                  'Generator loss: {}'.format(epoch,
                                              discriminator_loss.data.numpy(),
                                              generator_loss.data.numpy()))
            noise = torch.randn(len(data), noise_dim) # noise for fake sample
            fake_sample = generator(noise)
            # convert back from one-hot encoding
            fake_df = pd.DataFrame(enc.inverse_transform(fake_sample.detach()))
            fake_df.columns = df.columns
            show_heatmap(fake_df)

In [ ]:
train(data, generator, discriminator)

In [ ]:
# show real heatmap to compare
show_heatmap(df)

### Wasserstein GAN

In order to convert from a classic GAN to a Wasserstein GAN, we have to modify our discriminator network and our loss function. The discriminator will no longer return a probability, but instead a real value which can be thought of as a score for how realistic the data is. The discriminator wants to assign low scores to fake data and high scores to real data. The generator wants the discriminator to assign high scores to its data.

The loss function will no longer use cross-entropy. For the discriminator, the loss is the mean score for fake data minus the mean score for real data (thus the loss is low when the discriminator assigns low scores to fake data and high scores to real data). For the generator, the loss is minus the mean score for fake data (thus the loss is low when the generator creates data that the discriminator gives a high score).

Below is the code from above for the classic GAN. Modify the code to turn it into a Wasserstein GAN.

In [ ]:
generator = torch.nn.Sequential(
    torch.nn.Linear(noise_dim, hidden_dim),
    torch.nn.ReLU(),
    MultiCategoryGumbelSoftmax(hidden_dim, output_dims)
)

discriminator = torch.nn.Sequential(
    torch.nn.Linear(data_dim, hidden_dim),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(hidden_dim, 1),
    torch.nn.Sigmoid()
)

def train(data, generator, discriminator,
          epochs=20, n_discriminator=5, batch_size=128,
          learning_rate=1e-3):
    """Train the GAN
    
    Parameters
    ----------
    data : torch.Tensor
        Data for training
    generator : torch.nn.Sequential
        Generator network
    discriminator : torch.nn.Sequential
        Discriminator network
    epochs : int
        Number of iterations over the full data set for training
    n_discriminator : int
        Number of discriminator training iterations
    batch_size : int
        Number of training examples per inner iteration
    learning_rate : float
        Learning rate for training
    """
    # these solvers are optimizers for learning the parameters for our networks
    # we have one for each network (the generator and the discriminator)
    # RMSprop is one of many optimizers, and the choice is not terribly important here
    # The learning rate influences how large of a step the optimizer takes when it updates the parameters
    generator_solver = torch.optim.RMSprop(
        generator.parameters(), lr=learning_rate
    )
    
    discriminator_solver = torch.optim.RMSprop(
        discriminator.parameters(), lr=learning_rate
    )

    # There is a batch for each discriminator training iteration,
    # so each epoch is epoch_length iterations, and the total number of
    # iterations is the number of epochs times the length of each epoch.
    epoch_length = len(data) / (n_discriminator * batch_size)
    n_iters = int(epochs * epoch_length)
    
    # ones and zeros for calculating cross-entropy
    ones_label = torch.ones(batch_size, 1) # the real data should all be labeled 1
    zeros_label = torch.zeros(batch_size, 1) # the fake data should all be labeled 0
    
    # training loop
    for iteration in range(n_iters):
        for _ in range(n_discriminator):
            # Sample real data
            rand_perm = torch.randperm(data.size(0))
            real_sample = data[rand_perm[:batch_size]]

            # Sample fake data
            noise = torch.randn(batch_size, noise_dim)
            fake_sample = generator(noise)

            # Have the discriminator score the data
            discriminator_real = discriminator(real_sample)
            discriminator_fake = discriminator(fake_sample)

            # Calculate discriminator loss
            discriminator_loss = (
                # cross-entropy for real data (all should be 1 for ideal discriminator)
                F.binary_cross_entropy(discriminator_real, ones_label) +
                # cross-entropy for fake data (all should be 0 for ideal discriminator)
                F.binary_cross_entropy(discriminator_fake, zeros_label)
            )

            discriminator_loss.backward() # backpropagate the loss through the discriminator network
            discriminator_solver.step() # update the discriminator network parameters using the optimizer

            # Reset the gradients
            generator.zero_grad()
            discriminator.zero_grad()

        # Sample and score fake data
        noise = torch.randn(batch_size, noise_dim)
        fake_sample = generator(noise)
        discriminator_fake = discriminator(fake_sample)

        # Calculate generator loss
        # generator wants to trick the discriminator into thinking all of the fake data is real (labeled 1)
        generator_loss = F.binary_cross_entropy(discriminator_fake, ones_label)

        generator_loss.backward() # backpropagate the loss through the generator network
        generator_solver.step() # update the generator network parameters using the optimizer

        # Reset the gradients
        generator.zero_grad()
        discriminator.zero_grad()

        # Show training losses and sample crosstabs after each epoch
        if int(iteration % epoch_length) == 0:
            epoch = int(iteration / epoch_length)
            print('Epoch {}\n'
                  'Discriminator loss: {}; '
                  'Generator loss: {}'.format(epoch,
                                              discriminator_loss.data.numpy(),
                                              generator_loss.data.numpy()))
            noise = torch.randn(len(data), noise_dim) # noise for fake sample
            fake_sample = generator(noise)
            # convert back from one-hot encoding
            fake_df = pd.DataFrame(enc.inverse_transform(fake_sample.detach()))
            fake_df.columns = df.columns
            show_heatmap(fake_df)

## Private GANs

To ensure that our Wasserstein GAN is differentially-private, we need to make two modifications. First, we need to add Gaussian noise to every gradient used to update the parameters. Second, we need to clip or clamp the parameters (weights) to ensure they stay within a fixed range. The next two sections show the necessary functions in `PyTorch`.

### PyTorch hooks

Hooks let you apply a function to modify the gradient before it is used to update a parameter. This is helpful for us, because for differential privacy, we need to be able to add noise to every gradient step of training the discriminator. Here's some example code:

The following code can be used to get all of the parameters of the discriminator network:

In [ ]:
discriminator.parameters()

Note that this gives us a generator. This is all we need when writing our hooks, but if you want to see what the list of parameters really looks like, you can use a list comprehension to force Python to compute everything:

In [ ]:
[param for param in discriminator.parameters()]

We see a list of four parameters. The number of parameters in a given layer depends on the input and output dimensions, and the type of layer. `Linear` layers contain two parameters, a weight matrix and a bias vector (`Pytorch` uses $y=xA^T+b$ for the `Linear` calculation). The first linear layer has a `16x6` weight matrix (corresponding to the input dimension of 6 and output dimension of 16) and a `16`-dimensional bias vector. The second layer has a `1x16` weight matrix and a `1`-dimensional bias vector. Note that most non-linearities (`ReLU`, `sigmoid`, `tanh`) do not have any parameters, as they are fixed functions.

In [ ]:
# store the first parameter in a variable to illustrate registering a hook
parameter = next(discriminator.parameters())

In [ ]:
# this will add 1 to every gradient
parameter.register_hook(
    lambda grad: grad + 1
)

### Weight clamping
We also need to make sure our parameters stay within a fixed range to ensure differential privacy. PyTorch has a function `clamp_` which can be used on any parameter to clip its value to within a range. Notice in this example now no negative parameter values appear.

In [ ]:
parameter.data.clamp_(min=0, max=1)

You now have all the tools that you need to turn your Wasserstein GAN into a differentially-private Wasserstein GAN. First get your code working with just the gradient noise and use a very small amount of noise (e.g. `sigma = 0.01`). Then add the weight clip and use a large weight clip (e.g. `1`). If your GAN is still producing realistic outputs, experiment with what happens as you dial the noise (`sigma`) up or the weight clip down.

### Differentially-Private Wasserstein GAN

Below is the code for a functional Wasserstein GAN. Update the code to turn it into a DPWGAN.

In [ ]:
generator = torch.nn.Sequential(
    torch.nn.Linear(noise_dim, hidden_dim),
    torch.nn.ReLU(),
    MultiCategoryGumbelSoftmax(hidden_dim, output_dims)
)

discriminator = torch.nn.Sequential(
    torch.nn.Linear(data_dim, hidden_dim),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(hidden_dim, 1)
)

def train(data, generator, discriminator,
          epochs=20, n_discriminator=5, batch_size=128,
          learning_rate=1e-3):
    """Train the GAN
    
    Parameters
    ----------
    data : torch.Tensor
        Data for training
    generator : torch.nn.Sequential
        Generator network
    discriminator : torch.nn.Sequential
        Discriminator network
    epochs : int
        Number of iterations over the full data set for training
    n_discriminator : int
        Number of discriminator training iterations
    batch_size : int
        Number of training examples per inner iteration
    learning_rate : float
        Learning rate for training
    """
    # these solvers are optimizers for learning the parameters for our networks
    # we have one for each network (the generator and the discriminator)
    # RMSprop is one of many optimizers, and the choice is not terribly important here
    # The learning rate influences how large of a step the optimizer takes when it updates the parameters
    generator_solver = torch.optim.RMSprop(
        generator.parameters(), lr=learning_rate
    )
    
    discriminator_solver = torch.optim.RMSprop(
        discriminator.parameters(), lr=learning_rate
    )

    # There is a batch for each discriminator training iteration,
    # so each epoch is epoch_length iterations, and the total number of
    # iterations is the number of epochs times the length of each epoch.
    epoch_length = len(data) / (n_discriminator * batch_size)
    n_iters = int(epochs * epoch_length)
    
    # training loop
    for iteration in range(n_iters):
        for _ in range(n_discriminator):
            # Sample real data
            rand_perm = torch.randperm(data.size(0))
            real_sample = data[rand_perm[:batch_size]]

            # Sample fake data
            noise = torch.randn(batch_size, noise_dim)
            fake_sample = generator(noise)

            # Have the discriminator score the data
            discriminator_real = discriminator(real_sample)
            discriminator_fake = discriminator(fake_sample)

            # Calculate discriminator loss
            # Discriminator wants to assign a high score to real data
            # and a low score to fake data            
            discriminator_loss = -(
                torch.mean(discriminator_real) -
                torch.mean(discriminator_fake)
            )

            discriminator_loss.backward() # backpropagate the loss through the discriminator network
            discriminator_solver.step() # update the discriminator network parameters using the optimizer

            # Reset the gradients
            generator.zero_grad()
            discriminator.zero_grad()

        # Sample and score fake data
        noise = torch.randn(batch_size, noise_dim)
        fake_sample = generator(noise)
        discriminator_fake = discriminator(fake_sample)

        # Calculate generator loss
        # Generator wants discriminator to assign a high score to fake data
        generator_loss = -torch.mean(discriminator_fake)

        generator_loss.backward() # backpropagate the loss through the generator network
        generator_solver.step() # update the generator network parameters using the optimizer

        # Reset the gradients
        generator.zero_grad()
        discriminator.zero_grad()

        # Show training losses and sample crosstabs after each epoch
        if int(iteration % epoch_length) == 0:
            epoch = int(iteration / epoch_length)
            print('Epoch {}\n'
                  'Discriminator loss: {}; '
                  'Generator loss: {}'.format(epoch,
                                              discriminator_loss.data.numpy(),
                                              generator_loss.data.numpy()))
            noise = torch.randn(len(data), noise_dim) # noise for fake sample
            fake_sample = generator(noise)
            # convert back from one-hot encoding
            fake_df = pd.DataFrame(enc.inverse_transform(fake_sample.detach()))
            fake_df.columns = df.columns
            show_heatmap(fake_df)

Our flight example is quite simple, and you may be wondering if GANs can capture more complex data or patterns. They can, but the architecture of the generator or discriminator may need to be altered, and the training times will increase. In the `data` folder, you can find `pums_il.csv`, which contains real microdata from the US Census. If you would like to continue after this training, try playing with the network you've built and see if you can get it to produce realistic Census data. It shouldn't require much modification, but training the network will take much longer than our small example here.